# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.json

[
    {
        "title": "Down with the Hierarchy: The 'H' in HNSW stands for 'Hubs'",
        "pub_date": "2025-01-22", 
        "url_slug": "flatnav-paper", 
        "paper_url": "https://arxiv.org/pdf/2412.01940",
        "teaser_url": "",
        "venue": "(Preprint)", 
        "acceptance_rate": "",
        "acceptance_type": "",
        "citation": "<i>(Preprint)</i> <u>Blaise Munyampirwa</u>, Vihan Lakshman, Benjamin Coleman. <b>&quot;Down with the Hierarchy: The 'H' in HNSW stands for 'Hubs'&quot;</b>.",
        "excerpt": "Driven by recent breakthrough advances in neural representation learning, approximate near-neighbor (ANN) search over vector embeddings has emerged as a critical computational workload. With the introduction of the seminal Hierarchical Navigable Small World (HNSW) algorithm, graph-based indexes have established themselves as the overwhelmingly dominant paradigm for efficient and scalable ANN search. As the name suggests, HNSW searches a layered hierarchical 

## Import pandas

We are using the very handy pandas library for dataframes.

In [4]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [13]:
publications = pd.read_json("publications.json")
publications


,title,pub_date,url_slug,paper_url,teaser_url,venue,acceptance_rate,acceptance_type,alternative_url,citation,excerpt
0,Down with the Hierarchy: The 'H' in HNSW stand...,2025-01-22,flatnav-paper,https://arxiv.org/pdf/2412.01940,,(Preprint),,,,"<i>(Preprint)</i> <u>Blaise Munyampirwa</u>, V...",Driven by recent breakthrough advances in neur...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [11]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [14]:
import os
for row, item in publications.iterrows():
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    md = "---\ntitle: \""   + item.title + '"'
    md += "\ncollection: publications"
    md += f'\nurlslug: "{item.url_slug}"'
    md += """\npermalink: /publication/""" + html_filename
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    md += "\ndate: " + str(item.pub_date) 
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    if len(str(item.acceptance_rate)) > 0:
        md += "\nacceptancerate: '" + item.acceptance_rate + "'"
    if len(str(item.acceptance_type)) > 0:
        md += "\nacceptancetype: '" + item.acceptance_type + "'"
    if len(str(item.alternative_url)) > 5:
        md += "\nalternativeurl: '" + item.alternative_url + "'"
    if len(str(item.teaser_url)) > 5:
        md += "\nteaserurl: '" + item.teaser_url + "'"
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    md += "\n---"
    
    ## Markdown description for individual page
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    if len(str(item.paper_url)) > 5 and "internal" not in str(item.paper_url):
        md += "\n[Download](" + item.paper_url + ")\n"  
    md += f"\nCitation: {item.citation} "
    if len(str(item.acceptance_type)) > 0:
        md += f"({item.acceptance_type}) "
    if len(str(item.acceptance_rate)) > 0:
        md += f"(acceptance-rate: {item.acceptance_rate}) "
    md_filename = os.path.basename(md_filename)
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [15]:
!ls ../_publications/

2025-01-22-flatnav-paper.md


In [16]:
!cat ../_publications/2025-01-22-flatnav-paper.md

---
title: "Down with the Hierarchy: The 'H' in HNSW stands for 'Hubs'"
collection: publications
urlslug: "flatnav-paper"
permalink: /publication/2025-01-22-flatnav-paper
excerpt: 'Driven by recent breakthrough advances in neural representation learning, approximate near-neighbor (ANN) search over vector embeddings has emerged as a critical computational workload. With the introduction of the seminal Hierarchical Navigable Small World (HNSW) algorithm, graph-based indexes have established themselves as the overwhelmingly dominant paradigm for efficient and scalable ANN search. As the name suggests, HNSW searches a layered hierarchical graph to quickly identify neighborhoods of similar points to a given query vector. But is this hierarchy even necessary? A rigorous experimental analysis to answer this question would provide valuable insights into the nature of algorithm design for ANN search and motivate directions for future work in this increasingly crucial domain. To that end, we con